# Data processing and linear regression with Exasol

This tutorial is for users who want to use Exasol as a data source for training a linear regression model.
However, the data analyzing and processing part can also be interesting to anyone who's looking for the information how to prepare a dataset for a training. 

In this tutorial we will discuss the following topics:

- Part 1. How to import a dataset from CSV file to Exasol database.
- Part 2. How to analyze the data in the table using.
- Part 3. How to prepare data for using it in linear regression model.
- Part 4. How to create and train a model.

### Prerequisites

The users are assumed to have a basic understanding of Exasol, SQL and also basic Python programming knowledge.

## Part 1. Importing data

In our tutorial we use `Flights.csv` file which provides information about the U.S. domestic flights from 1987 to 2019 year.
The file contains more than 184 millions rows and 109 columns.

As the CSV file is too big, - more than 80 GB - we can't work with it directly importing is as a Pandas DataFrame.
That's the reason why we need to use a database as a warehouse and also a workbench for a future data transformation process.

So, the first step: **we start an Exasol Database**. 
For this tutorial we installed an Exasol Cloud Image using [Exasol Cloud Wizard](https://cloudtools.exasol.com/#/).
If you want to know more about Exasol Cloud Wizard, you can read an article about how to use this tool in our [blog](https://www.exasol.com/en/blog/building-clusters-in-the-sky-exasol-cloud-wizard/).
A local Exasol node was not a good option in our case, because a local machine with this node proceeds data significantly slower than the cloud image.

The second step: **import data from CSV file to Exasol's table**. 

1. We create an SQL CREATE TABLE statement and put it in a separate file named `flights.sql`.
Our table will have 109 columns according to the CSV file. Here is a part of the query:

In [ ]:
CREATE OR REPLACE TABLE "FLIGHTS" ("YEAR" INTEGER, "QUARTER" INTEGER, "MONTH" INTEGER, "DAY_OF_MONTH" INTEGER, "DAY_OF_WEEK" DECIMAL(1,0), ... , ...);


2. We connect to the Exasol, create a new schema and create a new table using the statement above.

 For connection and executing queries we use [pyexasol](https://github.com/badoo/pyexasol) library.


In [ ]:
import pyexasol

connection = pyexasol.connect(dsn='host:port', user='username', password='password', compression=True)

connection.execute("CREATE SCHEMA IF NOT EXISTS {schema_name};".format(schema_name="FLIGHTS"))
connection.open_schema("FLIGHTS")
create_table_query = open('flights.sql', 'r')
for line in create_table_query:
    connection.execute(query=line)

 3. After table is ready, we are importing the CSV file's content to the table using pyexasol. 
    The file is stored in Google Storage because of its size, so we only use a link to the file.

In [ ]:
connection.execute("IMPORT INTO {table_name} FROM CSV AT '{file_path}' FILE '{file_name}' "
            "COLUMN SEPARATOR = '{column_separator}' SKIP = 1  "
            "ERRORS INTO error_table (CURRENT_TIMESTAMP) REJECT LIMIT UNLIMITED ERRORS".format(
                table_name="FLIGHTS", file_path="https://storage.googleapis.com/our/path/", file_name="flights.single.csv.gz", column_separator=","))
connection.close()

Now you should be able to find all the data in the Exasol's table. 
Below you can see a python class that is doing the process described above.

In [ ]:
import simplestopwatch


class CsvImporter:
    def __init__(self, connection):
        self.connection = connection

    def import_file(self, sql_create_table_file, schema_name, table_name, file_path, file_name, column_separator):
        self.__handle_schema(self.connection, schema_name)
        self.__execute_create_table(sql_create_table_file)
        self.__run_import_command(self.connection, table_name, file_path, file_name, column_separator)

    def __execute_create_table(self, sql_create_table_file):
        query = open(sql_create_table_file, 'r')
        for line in query:
            self.connection.execute(query=line)

    def __handle_schema(self, connection, schema_name):
        connection.execute("CREATE SCHEMA IF NOT EXISTS " + schema_name + ";")
        connection.open_schema(schema_name)

    def __run_import_command(self, connection, table_name, file_path, file_name, column_separator):
        timer = simplestopwatch.Timer()
        connection.execute(
            "IMPORT INTO {table_name} FROM CSV AT '{file_path}' FILE '{file_name}' "
            "COLUMN SEPARATOR = '{column_separator}' SKIP = 1  "
            "ERRORS INTO error_table (CURRENT_TIMESTAMP) REJECT LIMIT UNLIMITED ERRORS".format(
                table_name=table_name, file_path=file_path, file_name=file_name, column_separator=column_separator))
        timer.stop()
        print("Imported in " + str(timer))